In [35]:
import pypyodbc as odbc
import pandas as pd
from credential import username,password
import pyodbc

In [36]:
#pip install pypyodbc
#!pip install credential
!pip install pyodbc

In [39]:
import pyodbc

conn_str = (
    "DRIVER=/opt/homebrew/Cellar/msodbcsql17/17.10.6.1/lib/libmsodbcsql.17.dylib;"
    "SERVER=localhost;"
    "DATABASE=Rishi;"     # Replace with your actual database name
    "UID=username;"             # Replace with your actual username
    "PWD=Password;"      # Replace with your actual password
    "Encrypt=yes;"
    "TrustServerCertificate=yes;"
)

try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    cursor.execute("SELECT 1")
    result = cursor.fetchone()
    if result:
        print("Connection successful! Result:", result)
    conn.close()

except pyodbc.Error as e:
    print("Error occurred:", e)


Connection successful! Result: (1, )


In [41]:
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0
orders.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [42]:
import zipfile
zip_ref = zipfile.ZipFile('orders.csv.zip') 
zip_ref.extractall() # extract file to dir
zip_ref.close() # close file


In [60]:
#read data from the file and handle null values
import pandas as pd
df = pd.read_csv('orders.csv',na_values=['Not Available','unknown'])
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [61]:
df.head(5)

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [62]:
df.columns=df.columns.str.lower()
df.columns=df.columns.str.replace(" ","_")

In [63]:
#derive new columns discount , sale price and profit

df['discount']=df['list_price']*df['discount_percent']*.01
df['sale_price']= df['list_price']-df['discount']
df['profit']=df['sale_price']-df['cost_price']

In [64]:
df.head(5)

,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent,discount,sale_price,profit
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2,5.2,254.8,14.8
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3,21.9,708.1,108.1
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5,0.5,9.5,-0.5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2,19.2,940.8,160.8
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5,1.0,19.0,-1.0


In [65]:
df.dtypes

order_id              int64
order_date           object
ship_mode            object
segment              object
country              object
city                 object
state                object
postal_code           int64
region               object
category             object
sub_category         object
product_id           object
cost_price            int64
list_price            int64
quantity              int64
discount_percent      int64
discount            float64
sale_price          float64
profit              float64
dtype: object

In [66]:
df['order_date']=pd.to_datetime(df['order_date'],format="%Y-%m-%d")

In [67]:
df.dtypes

order_id                     int64
order_date          datetime64[ns]
ship_mode                   object
segment                     object
country                     object
city                        object
state                       object
postal_code                  int64
region                      object
category                    object
sub_category                object
product_id                  object
cost_price                   int64
list_price                   int64
quantity                     int64
discount_percent             int64
discount                   float64
sale_price                 float64
profit                     float64
dtype: object

In [68]:
df.drop(columns=['list_price','cost_price','discount_percent'],inplace=True)

Inplace ---> drop in dataset

In [27]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [30]:
import sqlalchemy as sal
import pandas as pd

In [69]:
df = pd.DataFrame(df)

In [83]:
engine = create_engine("mssql+pyodbc:///?odbc_connect={}".format(conn_str))

# Upload the modified DataFrame to SQL Server
try:
    df.to_sql('df_orders', con=engine, index=False, if_exists='replace')
    print("DataFrame uploaded successfully!")
except Exception as e:
    print("Error uploading DataFrame:", e)

# Close the connection
engine.dispose()  # Closes the SQLAlchemy engine

DataFrame uploaded successfully!


** If we do if_exists='replace', pandas will assign highest possible data type while creating the table in the database. use 'append' instead.


In [72]:
df.columns

Index(['order_id', 'order_date', 'ship_mode', 'segment', 'country', 'city',
       'state', 'postal_code', 'region', 'category', 'sub_category',
       'product_id', 'quantity', 'discount', 'sale_price', 'profit'],
      dtype='object')

In [80]:
duplicates = df[df.duplicated(subset=['order_id'], keep=False)]
print(duplicates)


Empty DataFrame
Columns: [order_id, order_date, ship_mode, segment, country, city, state, postal_code, region, category, sub_category, product_id, quantity, discount, sale_price, profit]
Index: []


In [ ]:
engine = create_engine("mssql+pyodbc:///?odbc_connect={}".format(conn_str))

# Upload the modified DataFrame to SQL Server
try:
    df.to_sql('df_orders', con=engine, index=False, if_exists='append')
    print("DataFrame uploaded successfully!")
except Exception as e:
    print("Error uploading DataFrame:", e)

# Close the connection
engine.dispose()  # Closes the SQLAlchemy engine